🧱 **Célula 1 — Cabeçalho e Introdução**

"""

PROJETO: Análise e Automação de Vendas de um E-commerce Brasileiro

AUTOR: Gustavo de Paula Silva (@gustavogit4)

OBJETIVO:
    Este notebook gera um banco de dados relacional realista (clientes, produtos e vendas)
    para demonstrar habilidades em Python, SQL e Power BI, abordando conceitos de:
    
    - ETL e modelagem de dados;
    
    - simulação estatística e controle de reprodutibilidade;
    
    - geração de dados sintéticos representativos para análise exploratória e BI.

"""

**1️⃣ Importação das bibliotecas**

In [11]:
# ============================
# IMPORTAÇÃO DE BIBLIOTECAS
# ============================

# Importações principais
import os
import sys
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import date, datetime, timezone, timedelta
import sqlite3

print("✅ Bibliotecas importadas com sucesso!")

✅ Bibliotecas importadas com sucesso!


**2️⃣ Configuração de ambiente e caminhos**

In [12]:
def em_notebook():
    """Detecta se o código está sendo executado em um notebook."""
    try:
        shell = get_ipython().__class__.__name__
        if shell in ("ZMQInteractiveShell", "Shell"):  # VSCode, Colab, Jupyter
            return True
        else:
            return False
    except NameError:
        return False

# Define diretório base (compatível com notebook e script .py)
if em_notebook():
    BASE_DIR = os.getcwd()
else:
    BASE_DIR = os.path.dirname(os.path.abspath(__file__))

# Caminho para o banco de dados
CAMINHO_BANCO = os.path.join(BASE_DIR, "data", "ecommerce_realista.db")

# Cria a pasta 'data' se não existir
os.makedirs(os.path.join(BASE_DIR, "data"), exist_ok=True)

print(f"📂 Caminho do banco configurado em: {CAMINHO_BANCO}")

📂 Caminho do banco configurado em: c:\Users\--\Desktop\ecommerce-data-analytics\notebooks\data\ecommerce_realista.db


**3️⃣ Configurações do simulador**

In [13]:
# ============================
# CONFIGURAÇÕES DO SIMULADOR
# ============================

"""
Define os parâmetros principais da simulação e garante reprodutibilidade.
Todas as variáveis-chave estão centralizadas aqui para fácil ajuste.
"""

N_CLIENTES = 1000
N_PRODUTOS = 50
N_VENDAS = 15000
SEED = 42

np.random.seed(SEED)
random.seed(SEED)
faker = Faker("pt_BR")

print("⚙️ Configurações definidas com sucesso!")

⚙️ Configurações definidas com sucesso!


**4️⃣ Geração da tabela de clientes**

In [14]:
# ============================
# GERAÇÃO DA TABELA DE CLIENTES
# ============================

"""
Cria a tabela de clientes com informações demográficas e geográficas simuladas.
Adiciona variáveis derivadas que não alteram a estrutura original usada nos notebooks seguintes.
"""

from datetime import date

hoje = date.today()

idades = np.random.normal(35, 10, N_CLIENTES).astype(int)
idades = np.clip(idades, 18, 70)

clientes = pd.DataFrame({
    "id_cliente": range(1, N_CLIENTES + 1),
    "nome": [faker.name() for _ in range(N_CLIENTES)],
    "idade": idades,
    "genero": np.random.choice(["Masculino", "Feminino"], size=N_CLIENTES),
    "cidade": [faker.city() for _ in range(N_CLIENTES)],
    "estado": [faker.estado_sigla() for _ in range(N_CLIENTES)],
    "data_cadastro": [faker.date_between(start_date="-3y", end_date="today") for _ in range(N_CLIENTES)]
})

# -------- Variáveis complementares (sem alterar estrutura) --------
def faixa_etaria(idade):
    if idade < 25:
        return "18–24"
    elif idade < 35:
        return "25–34"
    elif idade < 50:
        return "35–49"
    else:
        return "50+"

clientes["faixa_etaria"] = clientes["idade"].apply(faixa_etaria)
clientes["tempo_cliente_dias"] = (pd.to_datetime(hoje) - pd.to_datetime(clientes["data_cadastro"])).dt.days
clientes["canal_preferencial"] = np.random.choice(
    ["E-commerce", "Loja Física", "WhatsApp"],
    size=N_CLIENTES,
    p=[0.45, 0.45, 0.10]
)

print("👥 Tabela 'clientes' revisada e gerada com sucesso!")
display(clientes.head(10))

👥 Tabela 'clientes' revisada e gerada com sucesso!


,id_cliente,nome,idade,genero,cidade,estado,data_cadastro,faixa_etaria,tempo_cliente_dias,canal_preferencial
0,1,Henry da Costa,39,Masculino,Pires,CE,2025-09-18,35–49,39,Loja Física
1,2,Srta. Evelyn da Mota,33,Feminino,Nascimento,PR,2024-09-17,25–34,405,E-commerce
2,3,Natália Pereira,41,Masculino,Ferreira da Serra,AC,2025-07-30,35–49,89,E-commerce
3,4,André Macedo,50,Feminino,Mendes,PE,2024-06-10,50+,504,E-commerce
4,5,Alícia Gonçalves,32,Masculino,da Rocha,TO,2024-10-25,25–34,367,WhatsApp
5,6,André Cirino,32,Masculino,Almeida,PB,2023-12-27,25–34,670,Loja Física
6,7,Pedro Henrique da Paz,50,Masculino,Carvalho,RN,2024-10-23,50+,369,E-commerce
7,8,Dr. Igor Gonçalves,42,Masculino,Monteiro da Prata,SC,2023-11-04,35–49,723,E-commerce
8,9,Luana Castro,30,Feminino,Rezende de da Rosa,RO,2023-12-18,25–34,679,E-commerce
9,10,Davi Luiz Sousa,40,Masculino,Abreu,AC,2025-05-19,35–49,161,Loja Física


**5️⃣ Geração da Tabela de Produtos**

In [15]:
# ============================
# 5️⃣ GERAÇÃO DA TABELA DE PRODUTOS
# ============================

"""
Cria a tabela de produtos com categorias distintas e faixas de preço realistas,
mantendo a estrutura original (id_produto, nome_produto, categoria, preco_unitario).
Inclui novas colunas opcionais (margem, estoque, fornecedor) que enriquecem o banco
para análises futuras no Power BI, sem afetar os notebooks anteriores.
"""

# Dicionário base de categorias e faixas de preço
categorias_preco = {
    "Alimentos": (5, 50),
    "Bebidas": (3, 80),
    "Limpeza": (5, 40),
    "Higiene": (8, 60),
    "Eletrônicos": (100, 3000)
}

# Lista de produtos brasileiros plausíveis
produtos_brasileiros = [
    "Arroz Tio João 5kg", "Feijão Carioca Camil 1kg", "Açúcar União 1kg", "Café Pilão 500g", "Macarrão Renata Espaguete 500g",
    "Leite Integral Itambé 1L", "Manteiga Aviação 200g", "Óleo de Soja Soya 900ml", "Biscoito Nestlé Recheado 140g", "Refrigerante Coca-Cola 2L",
    "Sabão em Pó Omo 1kg", "Amaciante Downy 1L", "Detergente Ypê 500ml", "Desinfetante Pinho Sol 500ml", "Água Sanitária Qboa 2L",
    "Shampoo Seda 325ml", "Condicionador Dove 325ml", "Sabonete Lux 90g", "Creme Dental Colgate 90g", "Desodorante Rexona Aerosol 150ml",
    "Notebook Lenovo IdeaPad 15\"", "Smartphone Samsung Galaxy A15", "Monitor LG 24\"", "Headset Logitech H390", "Mouse Sem Fio Dell WM126",
    "Teclado Mecânico Redragon Kumara", "Smart TV LG 50\"", "Caixa de Som JBL Flip 6", "Pen Drive Sandisk 64GB", "HD Externo Seagate 1TB",
    "Caderno Universitário Tilibra 10 Matérias", "Caneta Bic Azul", "Lápis Faber-Castell HB2", "Apontador Maped", "Borracha Mercur",
    "Mochila Dell Preto 15\"", "Agenda Permanente 2025", "Estojo Escolar Kipling", "Livro 'O Capital' – Karl Marx", "Livro 'Dom Casmurro' – Machado de Assis",
    "Vassoura Bettanin Noviça", "Rodo Sanremo 40cm", "Pano Multiuso Scotch-Brite", "Balde Plástico 10L", "Esponja de Aço Bombril",
    "Papel Higiênico Neve 12 Rolos", "Guardanapo Snob 50un", "Toalha de Papel Kitchen 2 Rolos", "Filtro de Café Melitta 103", "Pilha AA Duracell"
]

# Função para detectar categoria a partir do nome
def detectar_categoria(nome_produto):
    chaves = {
        "Alimentos": ["Arroz", "Feijão", "Açúcar", "Café", "Macarrão", "Leite", "Manteiga", "Óleo", "Biscoito"],
        "Bebidas": ["Refrigerante"],
        "Higiene": ["Shampoo", "Condicionador", "Sabonete", "Creme Dental", "Desodorante", "Papel Higiênico"],
        "Limpeza": ["Sabão", "Amaciante", "Detergente", "Desinfetante", "Água Sanitária", "Pano", "Vassoura", "Rodo"],
        "Eletrônicos": ["Notebook", "Smartphone", "Monitor", "Headset", "Mouse", "Teclado", "Smart TV", "Caixa de Som", "Pen Drive", "HD", "Pilha"]
    }
    for categoria, termos in chaves.items():
        if any(t.lower() in nome_produto.lower() for t in termos):
            return categoria
    return "Outros"

# Geração da tabela de produtos
produtos = pd.DataFrame({
    "id_produto": range(1, len(produtos_brasileiros) + 1),
    "nome_produto": produtos_brasileiros,
    "categoria": [detectar_categoria(p) for p in produtos_brasileiros],
})

# Atribui preços dentro da faixa de cada categoria
precos = []
for cat in produtos["categoria"]:
    min_p, max_p = categorias_preco.get(cat, (10, 100))
    precos.append(round(np.random.uniform(min_p, max_p), 2))
produtos["preco_unitario"] = precos

# --- Novas variáveis complementares (opcionais, não usadas no notebook 2) ---
# Margem de lucro estimada (%)
produtos["margem_lucro"] = np.where(produtos["categoria"] == "Eletrônicos",
                                    np.random.uniform(0.15, 0.30, len(produtos)),
                                    np.random.uniform(0.25, 0.45, len(produtos)))

# Estoque atual (simulado)
produtos["estoque_atual"] = np.random.randint(20, 500, size=len(produtos))

# Fornecedor simulado
produtos["fornecedor"] = np.random.choice(
    ["Distribuidora Global", "Atacado Minas", "Comercial Silva", "Fornecedor Nacional", "Importadora BRTech"],
    size=len(produtos)
)

print("🛍️ Tabela 'produtos' revisada com sucesso!")
display(produtos.head())

🛍️ Tabela 'produtos' revisada com sucesso!


,id_produto,nome_produto,categoria,preco_unitario,margem_lucro,estoque_atual,fornecedor
0,1,Arroz Tio João 5kg,Alimentos,8.29,0.357819,340,Fornecedor Nacional
1,2,Feijão Carioca Camil 1kg,Alimentos,33.01,0.306494,210,Atacado Minas
2,3,Açúcar União 1kg,Alimentos,49.15,0.335046,236,Distribuidora Global
3,4,Café Pilão 500g,Alimentos,13.55,0.257514,70,Comercial Silva
4,5,Macarrão Renata Espaguete 500g,Alimentos,40.67,0.275573,311,Comercial Silva


**6️⃣ Funções Auxiliares**

In [16]:
# ============================
# FUNÇÕES AUXILIARES
# ============================

"""
Define funções auxiliares utilizadas na geração da tabela de vendas.
Essas funções modelam comportamento temporal e preferências de consumo
sem alterar a estrutura original do banco de dados.
"""

from datetime import date

def gerar_data_venda():
    """
    Gera uma data de venda entre 2022 e 2024 com sazonalidade realista.

    As vendas são mais concentradas nos meses de:
    - Maio (Dia das Mães)
    - Novembro (Black Friday)
    - Dezembro (Natal)

    Retorna:
        datetime.date: Data da venda simulada.
    """
    data_inicio = date(2022, 1, 1)
    data_fim = date(2024, 12, 31)

    data = faker.date_between(start_date=data_inicio, end_date=data_fim)
    mes = data.month

    # Meses de pico com probabilidade maior de repetição
    if mes in [5, 11, 12] and random.random() < 0.6:
        ano = random.choice([2022, 2023, 2024])
        dia = random.randint(1, 28)
        return date(ano, mes, dia)

    return data


def canal_por_idade(idade):
    """
    Define o canal de venda com base na idade do cliente.

    Lógica:
    - Jovens (<30): preferência por E-commerce.
    - Adultos (30–49): equilíbrio entre loja física e online.
    - Idosos (50+): preferência por WhatsApp e loja física.

    Retorna:
        str: Canal de venda escolhido.
    """
    if idade < 30:
        return np.random.choice(
            ["E-commerce", "Loja Física", "WhatsApp"], p=[0.6, 0.3, 0.1]
        )
    elif idade < 50:
        return np.random.choice(
            ["E-commerce", "Loja Física", "WhatsApp"], p=[0.4, 0.5, 0.1]
        )
    else:
        return np.random.choice(
            ["E-commerce", "Loja Física", "WhatsApp"], p=[0.2, 0.4, 0.4]
        )


def aplicar_desconto(categoria, valor_total):
    """
    (Função opcional, não utilizada no Notebook 2)
    Simula políticas de desconto promocional para futuras análises no Power BI.

    Regras simuladas:
    - Eletrônicos: até 10%
    - Alimentos e Bebidas: até 5%
    - Demais categorias: até 8%

    Retorna:
        float: Valor total com desconto aplicado.
    """
    descontos = {
        "Eletrônicos": np.random.uniform(0, 0.10),
        "Alimentos": np.random.uniform(0, 0.05),
        "Bebidas": np.random.uniform(0, 0.05),
    }
    desconto = descontos.get(categoria, np.random.uniform(0, 0.08))
    return round(valor_total * (1 - desconto), 2)

**7️⃣ Geração da Tabela de Vendas**

In [17]:
# ============================
# GERAÇÃO DA TABELA DE VENDAS
# ============================

"""
Cria a tabela de vendas integrando as tabelas de clientes e produtos.
Mantém a estrutura original (id_venda, id_cliente, id_produto, quantidade,
data_venda, canal_venda, valor_total) — garantindo compatibilidade total
com as análises e dashboards existentes.

O modelo de geração considera:
- Clientes com frequência desigual de compra (exponencial)
- Categorias com diferentes volumes e ticket médio
- Canais de venda influenciados pela idade do cliente
- Sazonalidade anual (via gerar_data_venda)
"""

# --- Distribuição de clientes: alguns compram mais que outros ---
pesos = np.random.exponential(scale=0.8, size=N_CLIENTES)
pesos = pesos / pesos.sum()

clientes_vendas = np.random.choice(
    clientes["id_cliente"],
    size=N_VENDAS,
    p=pesos
)

# --- Dicionários auxiliares para lookup rápido ---
idades_clientes = clientes.set_index("id_cliente")["idade"].to_dict()
produtos_dict = produtos.set_index("id_produto")[["categoria", "preco_unitario"]].to_dict("index")

# --- Lista para armazenar as vendas simuladas ---
vendas = []

for i in range(N_VENDAS):
    id_cliente = int(clientes_vendas[i])
    idade_cliente = idades_clientes[id_cliente]
    id_produto = random.choice(list(produtos_dict.keys()))
    categoria = produtos_dict[id_produto]["categoria"]
    preco = produtos_dict[id_produto]["preco_unitario"]

    # Quantidade média varia conforme categoria
    if categoria == "Eletrônicos":
        quantidade = max(1, int(np.random.normal(1.2, 0.5)))
    elif categoria in ["Alimentos", "Bebidas"]:
        quantidade = max(1, int(np.random.normal(3, 1)))
    else:
        quantidade = max(1, int(np.random.normal(2, 0.8)))

    data_venda = gerar_data_venda()
    canal_venda = canal_por_idade(idade_cliente)

    # Valor total (sem aplicar desconto ainda)
    valor_total = round(quantidade * preco, 2)

    # --- Registro final da venda ---
    vendas.append({
        "id_venda": i + 1,
        "id_cliente": id_cliente,
        "id_produto": id_produto,
        "quantidade": quantidade,
        "data_venda": data_venda,
        "canal_venda": canal_venda,
        "valor_total": valor_total
    })

# --- Cria o DataFrame final de vendas ---
vendas = pd.DataFrame(vendas)

# --- Inserir 1% de valores nulos para testes de limpeza (mantido) ---
for col in ["canal_venda", "quantidade"]:
    nulos = int(len(vendas) * 0.01)
    vendas.loc[vendas.sample(n=nulos).index, col] = np.nan

print("💸 Tabela 'vendas' gerada com sucesso!")
display(vendas.head())

print(f"Total de registros: {len(vendas):,}")
print(f"Período de vendas: {vendas['data_venda'].min()} → {vendas['data_venda'].max()}")


💸 Tabela 'vendas' gerada com sucesso!


,id_venda,id_cliente,id_produto,quantidade,data_venda,canal_venda,valor_total
0,1,780,41,1.0,2022-03-05,WhatsApp,39.76
1,2,244,8,3.0,2024-01-05,E-commerce,144.63
2,3,963,2,2.0,2023-03-05,E-commerce,66.02
3,4,343,48,3.0,2022-11-05,E-commerce,286.74
4,5,338,48,2.0,2024-10-04,E-commerce,191.16


Total de registros: 15,000
Período de vendas: 2022-01-01 → 2024-12-30


**8️⃣ Validação dos Dados**

In [18]:
# ============================================
# VALIDAÇÃO DOS DADOS 
# ============================================

"""
Valida a integridade e consistência das tabelas antes de salvar o banco.

Essa etapa garante que o dataset está limpo, coerente e pronto para uso
nas análises e dashboards. Inclui verificações de nulos, tipos, coerência
quantitativa e estatísticas descritivas resumidas.
"""

print("🔎 Iniciando validação dos dados...\n")

# --- Contagem geral de registros ---
print("📊 Quantidade de registros:")
print(f"- Clientes:  {len(clientes):>6}")
print(f"- Produtos:  {len(produtos):>6}")
print(f"- Vendas:    {len(vendas):>6}\n")

# --- Verificação de valores nulos ---
print("🧩 Valores nulos por coluna (tabela vendas):")
nulos_vendas = vendas.isnull().sum()
print(nulos_vendas.to_string(), "\n")

# --- Tipos de dados e amostra ---
print("📋 Estrutura da tabela 'vendas':")
print(vendas.dtypes.to_string(), "\n")

print("🔎 Amostra de registros:")
display(vendas.head(5))

# --- Regras de coerência ---
print("\n🧠 Verificações de integridade:")

# Quantidade inválida
vendas_invalidas = vendas[vendas["quantidade"] <= 0]
if len(vendas_invalidas) > 0:
    print(f"⚠️  {len(vendas_invalidas)} registros com quantidade inválida detectados.")
else:
    print("✅ Nenhuma quantidade inválida encontrada.")

# Valor total inconsistente (preço unitário * quantidade ≈ valor_total)
verificacao_precos = vendas.merge(produtos[["id_produto", "preco_unitario"]], on="id_produto", how="left")
verificacao_precos["check"] = np.isclose(
    verificacao_precos["valor_total"],
    verificacao_precos["quantidade"] * verificacao_precos["preco_unitario"],
    atol=0.1
)
erros_preco = verificacao_precos[~verificacao_precos["check"]]
if len(erros_preco) > 0:
    print(f"⚠️  {len(erros_preco)} inconsistências de valor detectadas.")
else:
    print("✅ Todos os valores de venda estão coerentes.\n")

# --- Estatísticas resumidas ---
print("📈 Resumo estatístico de valores monetários:")
display(vendas["valor_total"].describe(percentiles=[0.25, 0.5, 0.75]).round(2))

# --- Intervalo temporal das vendas ---
print("🗓️ Intervalo de datas das vendas:")
print(f"Início: {vendas['data_venda'].min()}  |  Fim: {vendas['data_venda'].max()}\n")

# --- Caminho do banco (para conferência visual e conexão Power BI) ---
CAMINHO_BANCO = os.path.join(BASE_DIR, "data", "ecommerce_realista.db")
print(f"📂 Caminho configurado: {CAMINHO_BANCO}")


🔎 Iniciando validação dos dados...

📊 Quantidade de registros:
- Clientes:    1000
- Produtos:      50
- Vendas:     15000

🧩 Valores nulos por coluna (tabela vendas):
id_venda         0
id_cliente       0
id_produto       0
quantidade     150
data_venda       0
canal_venda    150
valor_total      0 

📋 Estrutura da tabela 'vendas':
id_venda         int64
id_cliente       int64
id_produto       int64
quantidade     float64
data_venda      object
canal_venda     object
valor_total    float64 

🔎 Amostra de registros:


,id_venda,id_cliente,id_produto,quantidade,data_venda,canal_venda,valor_total
0,1,780,41,1.0,2022-03-05,WhatsApp,39.76
1,2,244,8,3.0,2024-01-05,E-commerce,144.63
2,3,963,2,2.0,2023-03-05,E-commerce,66.02
3,4,343,48,3.0,2022-11-05,E-commerce,286.74
4,5,338,48,2.0,2024-10-04,E-commerce,191.16



🧠 Verificações de integridade:
✅ Nenhuma quantidade inválida encontrada.
⚠️  150 inconsistências de valor detectadas.
📈 Resumo estatístico de valores monetários:


count    15000.00
mean       353.36
std        685.20
min          6.57
25%         41.31
50%         86.92
75%        156.50
max       5571.92
Name: valor_total, dtype: float64

🗓️ Intervalo de datas das vendas:
Início: 2022-01-01  |  Fim: 2024-12-30

📂 Caminho configurado: c:\Users\--\Desktop\ecommerce-data-analytics\notebooks\data\ecommerce_realista.db


**9️⃣ Criação e Salvamento do Banco SQLite**

In [19]:
# ============================================
# CRIAÇÃO E SALVAMENTO DO BANCO SQLITE
# ============================================

"""
Cria (ou substitui) o banco de dados SQLite com as tabelas 'clientes',
'produtos', 'vendas' e 'atualizacao'. Garante compatibilidade total com
os notebooks seguintes e com a integração Power BI.

Formato: data/ecommerce_realista.db
"""

# --- Caminho final dentro do projeto local ---
CAMINHO_BANCO = os.path.join(BASE_DIR, "data", "ecommerce_realista.db")

# Cria a pasta 'data' se não existir
os.makedirs(os.path.dirname(CAMINHO_BANCO), exist_ok=True)

# --- Conexão com SQLite ---
conn = sqlite3.connect(CAMINHO_BANCO)

# Salva as tabelas principais
clientes.to_sql("clientes", conn, if_exists="replace", index=False)
produtos.to_sql("produtos", conn, if_exists="replace", index=False)
vendas.to_sql("vendas", conn, if_exists="replace", index=False)

# --- Registro de atualização ---
fuso_brasilia = timezone(timedelta(hours=-3))
agora_brasilia = datetime.now(fuso_brasilia)
data_atualizacao = pd.DataFrame({
    "ultima_atualizacao": [agora_brasilia.strftime("%Y-%m-%d %H:%M:%S")]
})
data_atualizacao.to_sql("atualizacao", conn, if_exists="replace", index=False)

# --- Confirma e fecha ---
conn.commit()
conn.close()

# --- Informações de verificação ---
tamanho_mb = os.path.getsize(CAMINHO_BANCO) / (1024 * 1024)
print("✅ Banco de dados SQLite criado e atualizado com sucesso!")
print(f"📂 Localização: {CAMINHO_BANCO}")
print(f"📊 Tamanho aproximado: {tamanho_mb:.2f} MB")
print(f"⏰ Última atualização: {agora_brasilia.strftime('%d/%m/%Y %H:%M:%S')} (Horário de Brasília)")


✅ Banco de dados SQLite criado e atualizado com sucesso!
📂 Localização: c:\Users\--\Desktop\ecommerce-data-analytics\notebooks\data\ecommerce_realista.db
📊 Tamanho aproximado: 0.78 MB
⏰ Última atualização: 27/10/2025 09:26:40 (Horário de Brasília)


**🔟 Teste de Conexão e Verificação de Estrutura**

In [20]:
# ============================================
# TESTE DE CONEXÃO AO BANCO DE DADOS
# ============================================

"""
Valida se o banco SQLite foi criado corretamente e está pronto para ser
lido por ferramentas externas, como Power BI ou scripts Python.

Essa célula:
1. Conecta ao banco local.
2. Lista as tabelas disponíveis.
3. Exibe uma amostra de registros de cada tabela.
"""

import sqlite3

# Conecta ao banco
conn = sqlite3.connect(CAMINHO_BANCO)

# --- 1. Listagem das tabelas ---
print("📚 Tabelas disponíveis no banco:")
tabelas = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
display(tabelas)

# --- 2. Visualização rápida das tabelas principais ---
for tabela in ["clientes", "produtos", "vendas"]:
    print(f"\n🔍 Amostra da tabela '{tabela}':")
    try:
        amostra = pd.read_sql_query(f"SELECT * FROM {tabela} LIMIT 5;", conn)
        display(amostra)
    except Exception as e:
        print(f"⚠️ Erro ao ler '{tabela}': {e}")

# --- 3. Confirma data de atualização ---
print("\n🕒 Última atualização registrada:")
try:
    ultima = pd.read_sql_query("SELECT * FROM atualizacao;", conn)
    display(ultima)
except Exception as e:
    print("⚠️ Não foi possível carregar a tabela de atualização:", e)

conn.close()
print("\n✅ Conexão encerrada com sucesso.")

📚 Tabelas disponíveis no banco:


,name
0,clientes
1,produtos
2,vendas
3,atualizacao



🔍 Amostra da tabela 'clientes':


,id_cliente,nome,idade,genero,cidade,estado,data_cadastro,faixa_etaria,tempo_cliente_dias,canal_preferencial
0,1,Henry da Costa,39,Masculino,Pires,CE,2025-09-18,35–49,39,Loja Física
1,2,Srta. Evelyn da Mota,33,Feminino,Nascimento,PR,2024-09-17,25–34,405,E-commerce
2,3,Natália Pereira,41,Masculino,Ferreira da Serra,AC,2025-07-30,35–49,89,E-commerce
3,4,André Macedo,50,Feminino,Mendes,PE,2024-06-10,50+,504,E-commerce
4,5,Alícia Gonçalves,32,Masculino,da Rocha,TO,2024-10-25,25–34,367,WhatsApp



🔍 Amostra da tabela 'produtos':


...


🔍 Amostra da tabela 'vendas':


,id_venda,id_cliente,id_produto,quantidade,data_venda,canal_venda,valor_total
0,1,780,41,1.0,2022-03-05,WhatsApp,39.76
1,2,244,8,3.0,2024-01-05,E-commerce,144.63
2,3,963,2,2.0,2023-03-05,E-commerce,66.02
3,4,343,48,3.0,2022-11-05,E-commerce,286.74
4,5,338,48,2.0,2024-10-04,E-commerce,191.16



🕒 Última atualização registrada:


,ultima_atualizacao
0,2025-10-27 09:26:40



✅ Conexão encerrada com sucesso.
